In [ ]:
import pandas as pd
import os

INPUT_FILE = "CosmosDB/scheduled_tests_qlearning.csv"  
OUTPUT_FILE = "CosmosDB/scheduled_tests_SPT.csv"
MAX_TOTAL_TIME = 20  # seconds (time budget)

# Load the data 
df = pd.read_csv(INPUT_FILE)

# Rename duration column 
if "duration_sec" in df.columns:
    df.rename(columns={"duration_sec": "execTime"}, inplace=True)

#Sort by execution time 
df_spt = df.sort_values(by="execTime").reset_index(drop=True)

# Accumulate execution time 
df_spt["cumulative_time"] = df_spt["execTime"].cumsum()

#Select tests within the time budget 
df_spt_scheduled = df_spt[df_spt["cumulative_time"] <= MAX_TOTAL_TIME].copy()

#Save the result 
os.makedirs("CosmosDB", exist_ok=True)
df_spt_scheduled.to_csv(OUTPUT_FILE, index=False)

# Print Summary 
print("SPT schedule saved to:", OUTPUT_FILE)
print(df_spt_scheduled[["testCase_id", "testcase_name", "execTime"]])
print(f"\n Total Scheduled Test Cases: {len(df_spt_scheduled)}")
print(f"⏱ Total Execution Time: {df_spt_scheduled['execTime'].sum():.2f} sec")




SPT schedule saved to: CosmosDB/scheduled_tests_SPT.csv
    testCase_id                                     testcase_name  execTime
0            12          PaintBotFaultyMove_ShouldFail {Autobot1}      0.00
1            11            PaintBotEdgeTolerance_Check {Autobot1}      0.00
2            14        SmartPaintBot_PositionLimitFail {Autobot1}      0.13
3            17  RoboticPainter_FailsafeBlocksPainting {Autobot2}      0.27
4            16   ABBAdvancedPaintBot_FailsafeMovement {Autobot1}      0.37
5            21          ABBIRB6000_MovementBoundaries {Autobot2}      0.46
6             9        ABBPaintBot_TriggerFailsafeFlow {Autobot1}      0.57
7            15             RoboticPaintX_MoveAndSpray {Autobot1}      0.60
8            19       ABBFlexiblePaintBot_LowEdgeCheck {Autobot2}      0.76
9             6          PaintIRB5500_ResetAfterFailure{Autobot1}      0.77
10           34        ABBPaintMaster_ResetClearsState {Autobot3}      0.92
11           35          ABBPain

In [12]:
import pandas as pd
import os

# --- Step 1: Load your CSV file ---
df = pd.read_csv("CosmosDB/baseline_input.csv")  # Replace with actual filename

# --- Step 2: Clean and preprocess ---
df = df[df["duration_sec"].notna()]  # Remove rows with missing durations

# ✅ Keep only the first occurrence (shortest duration) per unique test case
df = df.sort_values(by="duration_sec")
df_unique = df.drop_duplicates(subset="testCase_id", keep="first").reset_index(drop=True)

# --- Step 3: SPT Scheduling with 20-second limit ---
MAX_TIME = 20.0  # seconds
selected_tests = []
total_time = 0.0

for _, row in df_unique.iterrows():
    exec_time = row["duration_sec"]
    if total_time + exec_time <= MAX_TIME:
        selected_tests.append(row)
        total_time += exec_time
    else:
        break

# --- Step 4: Save the schedule ---
scheduled_df = pd.DataFrame(selected_tests)
os.makedirs("CosmosDB", exist_ok=True)
scheduled_df.to_csv("CosmosDB/scheduled_tests_spt.csv", index=False)

# --- Step 5: Summary output ---
print("✅ SPT Schedule Saved to 'CosmosDB/scheduled_tests_spt.csv'")
print(scheduled_df[["testCase_id", "testcase_name", "duration_sec"]].head(20))
print(f"🧪 Total Scheduled Test Cases: {len(scheduled_df)}")
print(f"⏱ Total Execution Time: {total_time:.2f} sec")


✅ SPT Schedule Saved to 'CosmosDB/scheduled_tests_spt.csv'
    testCase_id                                     testcase_name  \
0             9        ABBPaintBot_TriggerFailsafeFlow {Autobot1}   
1            26       ABBPaintingBot_RecoveryFromCrash {Autobot2}   
2            24     MotionSystemCheck_FailureDuringRun {Autobot2}   
3            25              CompactPaintBot_QuickTest {Autobot2}   
4            23          PaintLineIRB_ValidPositioning {Autobot2}   
5             7                  SprayRobot_ResetWorks {Autobot1}   
6            12          PaintBotFaultyMove_ShouldFail {Autobot1}   
7            21          ABBIRB6000_MovementBoundaries {Autobot2}   
8            22           IndustrialPaintBot_ValidStop {Autobot2}   
9            14        SmartPaintBot_PositionLimitFail {Autobot1}   
10            8                     ABB5000_PaintCycle {Autobot1}   
11           16   ABBAdvancedPaintBot_FailsafeMovement {Autobot1}   
12            5           PaintABB5400_Motio